<a href="https://colab.research.google.com/github/daryoush/minGPT/blob/master/startjulia1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%cd /gdrive/My Drive/
! git clone https://github.com/daryoush/minGPT.git

#! git pull
#! pip install <desired-python-library>

[Errno 2] No such file or directory: 'gdrive/My Drive/'
/content/gdrive/My Drive
Cloning into 'minGPT'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 198 (delta 6), reused 14 (delta 1), pack-reused 176
Receiving objects: 100% (198/198), 1.84 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (107/107), done.


# New Section

In [2]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
    "Random","InformationMeasures", "DataStructures", "Transformers","Statistics", "Dates",\
    "BSON",\
    "Images",\
    "MLDatasets",\
    "Parameters",\
    "Printf",\
    ]);' ###Pkg.precompile()'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed VersionParsing ── v1.2.0
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed SoftGlobalScope ─ v1.1.0
  Installed Conda ─────────── v1.5.0
  Installed ZMQ ───────────── v1.2.1
  Installed JSON ──────────── v0.21.1
  Installed IJulia ────────── v1.23.1
  Installed JLLWrappers ───── v1.1.3
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed MbedTLS ───────── v1.0.3
  Installed Parsers ───────── v1.0.15
######################################################################## 100.0%
######################################################################## 100.0%
Updating `~/.julia/environments/v1.5/Project.toml`
  [7073ff75] + IJulia v1.23.1
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [56f22d72] + Artifacts v1.3.0
  

**NOW GO TO EDIT NOTEBOOK SETTING AND SET IT TO JULIA**
For adding new packages, it should be in python or the syntax may need to be in Julia!

In [1]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
import Pkg; Pkg.add("BenchmarkTools")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [3]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     434.406 ms (0.00% GC)
  median time:      445.238 ms (0.00% GC)
  mean time:        461.517 ms (4.05% GC)
  maximum time:     539.216 ms (18.80% GC)
  --------------
  samples:          11
  evals/sample:     1

In [4]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  11
  --------------
  minimum time:     3.070 ms (0.00% GC)
  median time:      4.257 ms (0.00% GC)
  mean time:        4.298 ms (0.03% GC)
  maximum time:     8.426 ms (17.32% GC)
  --------------
  samples:          1162
  evals/sample:     1

In [5]:
cd("/content/gdrive/MyDrive/minGPT")
pwd()

"/content/gdrive/MyDrive/minGPT"

In [6]:
using Flux: onehot
	using Transformers: Vocabulary, decode
	using Random
	using Flux: throttle, @epochs, ADAMW, ADAM, RADAM, Momentum, Optimiser, logitcrossentropy, params
	using Flux: softmax, onecold
	using DataStructures: Queue, enqueue!, dequeue!
	Random.seed!(42)


MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

In [7]:
import Pkg; Pkg.add("NNlib")
Pkg.add("Distributions")
Pkg.add("Juno")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [11]:
using Flux

module Datasets include("onJulia/src/datasets.jl") end
dataSetlength, vocab, sample = Datasets.openDataset("input.txt")
batch(dataLength,batchLength ) = hcat((sample(dataLength) for x in 1:batchLength)...)

## ---
module Model include("onJulia/src/model.jl") end

lengthOfSentences=520
model=Model.minGPT(maxBlockSize=lengthOfSentences, vocabSize=length(vocab)) |> gpu


## ---


# for a batch of sentences, extract 1st to one before last character as input
# and 2nd to last char of each sentence as prediction.  Shape the predictred
# values as a long one dim series of values, one hot encode them using the vocab

trainingDataPair(b)=(b[1:end-1, :], onehot(vocab, reshape(b[2:end,:], :)))
trainingData(dataLength,batchLength)= [trainingDataPair(batch(dataLength,batchLength))
		for i in 1:dataSetlength/(dataLength*batchLength)]

function loss(x,y)
	yhat=model(x)
	logitcrossentropy(reshape(yhat,size(yhat)[1], :), y)
end


function validate()
	s=collect("Wher")
	for i in 1:20
		out=model(vocab(s))
		push!(s, decode(vocab,onecold(softmax(out)[:,end])))
	end
	join(s)
end

datasize = length(s) = 1115394


validate (generic function with 1 method)

In [15]:
opt = RADAM()

## ---
module Train include("onJulia/src/train.jl") end

Main.Train

In [13]:


function sampleFromModel(s="Whe", minprob=.05) 
	function explore(x, minprob)
		x[1] < minprob && return currentRes
		out=model(vocab(collect(x[2])))
		choices=sort(collect(zip(softmax(out[:,end]), decode(vocab, collect(1:size(out)[1])))), rev=true)
		options = [(x[1]*np, x[2]*nc) for (np,nc) in choices if (x[1]*np) > minprob ]
	end
	toexplore=Queue{Tuple{Float32,String}}()
	enqueue!(toexplore,(1, s))  #start with the initial string and prob 1
	results=[]
	for i in toexplore
		nxt=dequeue!(toexplore)
		res=explore(nxt, minprob)
		if isempty(res)  # go as long as you can't add any more character and satisfy the prob
			push!(results, nxt)
			continue #continue emptying the queue
		end
		for r in res
			enqueue!(toexplore, r)  ## add elements for further exploration
		end
	end
    #elements of the results are (prob, string), sort from high to low of the 
	#longest match with highest prob
	sort(results, by=x->(length(x[2]), x[1]), rev=true)
end


sampleFromModel (generic function with 3 methods)

In [23]:
sampleFromModel("Where i", 0.01)

44-element Array{Any,1}:
 (0.038411804f0, "Where iE")
 (0.037242673f0, "Where ip")
 (0.03473805f0, "Where id")
 (0.030332904f0, "Where iB")
 (0.0300927f0, "Where i.")
 (0.027924122f0, "Where ir")
 (0.027752897f0, "Where iT")
 (0.026616493f0, "Where iD")
 (0.026137497f0, "Where iR")
 (0.02282904f0, "Where i,")
 (0.022320388f0, "Where iM")
 (0.02206444f0, "Where i?")
 (0.021302607f0, "Where iF")
 ⋮
 (0.01326453f0, "Where iH")
 (0.013256247f0, "Where iN")
 (0.013218637f0, "Where i ")
 (0.012600855f0, "Where iK")
 (0.012550184f0, "Where i□")
 (0.01228842f0, "Where ib")
 (0.011988522f0, "Where it")
 (0.0113271875f0, "Where iy")
 (0.011017294f0, "Where i\n")
 (0.010945341f0, "Where iQ")
 (0.010677595f0, "Where i!")
 (0.010569763f0, "Where iW")

In [36]:


numberOfSentencesInBatch=10
for i in 1:10 
  d=trainingData(lengthOfSentences, numberOfSentencesInBatch) |> gpu
  Train.train!(loss,
            params(model),
            d,
            opt,
            sampleFromModel )
end



ctr = 0
(ctr, training_loss) = (0, 1.4958391f0)
training_accuracy = validate() = Any[(0.05878298f0, "Where's "), (0.2201425f0, "Where "), (0.19880947f0, "When "), (0.06879854f0, "Whe ")]
ctr = 1
ctr = 2
ctr = 3
ctr = 4
ctr = 5
ctr = 6
ctr = 7
ctr = 8
ctr = 9
ctr = 10
(ctr, training_loss) = (10, 1.5050296f0)
training_accuracy = validate() = Any[(0.058437172f0, "Where's "), (0.056571905f0, "Where i"), (0.2731889f0, "When ")]
ctr = 11
ctr = 12
ctr = 13
ctr = 14
ctr = 15
ctr = 16
ctr = 17
ctr = 18
ctr = 19
ctr = 20
(ctr, training_loss) = (20, 1.4521098f0)
training_accuracy = validate() = Any[(0.05472161f0, "Where is "), (0.13028507f0, "Where's "), (0.26251698f0, "When ")]
ctr = 21
ctr = 22
ctr = 23
ctr = 24
ctr = 25
ctr = 26
ctr = 27
ctr = 28
ctr = 29
ctr = 30
(ctr, training_loss) = (30, 1.5293752f0)
training_accuracy = validate() = Any[(0.14346856f0, "Where's "), (0.058223423f0, "When t")]
ctr = 31
ctr = 32
ctr = 33
ctr = 34
ctr = 35
ctr = 36
ctr = 37
ctr = 38
ctr = 39
ctr = 40
(ctr, trai

In [40]:
Pkg.add("BSON")
using BSON

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [43]:
using BSON:  @save, @load
@save "inputtext_model.bson"  model

In [45]:
using Zygote
using Flux
using Transformers
@load "inputtext_model.bson"  model

In [46]:
sampleFromModel("Can you be h", 0.001)


291-element Array{Any,1}:
 (0.0016024503f0, "Can you be here, that ")
 (0.0010629784f0, "Can you be here, they ")
 (0.0010095453f0, "Can you be heard with ")
 (0.0054474054f0, "Can you be here, and ")
 (0.0018994029f0, "Can you be heard and ")
 (0.001652218f0, "Can you be heard the ")
 (0.0016485763f0, "Can you be here that ")
 (0.0016252969f0, "Can you be here with ")
 (0.001262f0, "Can you be here,\nAnd ")
 (0.001223729f0, "Can you be here, but ")
 (0.0011988198f0, "Can you be heard you ")
 (0.0011876631f0, "Can you be here, the ")
 (0.0011736538f0, "Can you be here, you ")
 ⋮
 (0.001415358f0, "Can you be harm")
 (0.0013797823f0, "Can you be he, ")
 (0.0012552247f0, "Can you be hung")
 (0.0012375491f0, "Can you be huma")
 (0.001114813f0, "Can you be her!")
 (0.0010842477f0, "Can you be he a")
 (0.0010725647f0, "Can you be him!")
 (0.0015641853f0, "Can you be he\n")
 (0.0011478838f0, "Can you be hin")
 (0.0011136978f0, "Can you be hid")
 (0.001088511f0, "Can you be ham")
 (0.001176429

In [20]:
yPkg.add("StatsBase")
Pkg.add("Plots")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
  Installed Ogg_jll ───────── v1.3.4+2
  Installed LAME_jll ──────── v3.100.0+3
  Installed x265_jll ──────── v3.0.0+3
  Installed PlotThemes ────── v1.0.3
  Installed RecipesPipeline ─ v0.1.13
  Installed Plots ─────────── v1.0.14
  Installed Contour ───────── v0.5.7
  Installed GeometryTypes ─── v0.8.4
  Installed Showoff ───────── v0.3.2
  Installed Grisu ─────────── v1.0.0
  Installed FreeType2_jll ─── v2.10.1+5
  Installed Opus_jll ──────── v1.3.1+3
  Installed FFMPEG_jll ────── v4.3.1+4
  Installed LibVPX_jll ────── v1.9.0+1
  Installed libfdk_aac_jll ── v0.1.6+4
  Installed PlotUtils ─────── v0.6.5
  Installed libvorbis_jll ─── v1.3.6+6
  Installed FFMPEG ────────── v0.3.0
  Installed Bzip2_jll ─────── v1.0.6+5
  Installed Measures ──────── v0.3.1
  Installed FriBidi_jll ───── v1.0.5+6
  Installed libass_

In [21]:
using StatsBase
using Plots
function getOutputResults(m, data, limit=5)
	output=[]
	ctr=0
	for (x,y) in data
		out=softmax(m(x))
		out=reshape(out,size(out)[1], :)
		push!(output, onecold(out)...)  ## get the encoded vocab using straight onecold, not onecold(vocab, ..) version

		ctr +=1
		if  ctr > limit break end
	end
	convert(Array{Int}, output)
end

d=trainingData(lengthOfSentences, numberOfSentencesInBatch)
function OutputHistogram(m)
	outChars=getOutputResults(m, d)
	fit(Histogram, outChars, 0:1:length(vocab))
end

#Try a untrained model
raw=OutputHistogram(Model.minGPT(maxBlockSize=lengthOfSentences, vocabSize=length(vocab)))

#..a. and trained model
trained=OutputHistogram(model)

plot(raw)
plot!(trained)

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


LoadError: ignored